In [37]:
from modules import Path 

from tika import parser
import string
import re
import io
import os


# In dieser Datei soll eine Neue Datei geschrieben.
# Für jeden Jahresabschluss, den diese Datei einließt, soll eine Excel-Arbeitsmappe erstellt 
# werden, die anhand einer KeyWordListe matches abgleicht und den Satz des Matches extrahiert. 

#=============================================================================================
# 0. PFADE SAMMELN
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")
for pfad in pfade:
    print(pfad)

C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\BMW AG2008_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Deutsche Post AG2010_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\E.ON AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Evonik Industries AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Fraport AG FFM Airport2012_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Henkel AG_Co.KGAA2013_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Linde AG_SR_2014.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\MAN SE_SR_2015.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\RWE AG_SR_2016.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\t

In [38]:
# KWL laden:
KWL_raw = []
with open("KWL.txt", "r", encoding="utf-8") as KWL_file:
    for line in KWL_file:
        KWL_raw.append(line.strip())
len(KWL_raw)

70

In [39]:
# wegen dopplungen in der KWL-raw wird diese überarbeitet. 
KWL_set = set()
KWL = []

for key in KWL_raw:
    KWL_set.add(key)
for key in KWL_set:
    KWL.append(key)
len(KWL)

60

In [40]:
KWL

['aktualisieren',
 'revidiert',
 'angepasst',
 'Abänderung',
 'Bereinigung',
 'korrigieren',
 'ändern',
 'bearbeitet',
 'anpassen',
 'aktualisiert',
 'Ausbesserung',
 'bereinigt',
 'bereinigen',
 'abändern',
 'Überprüfung',
 'richtig stellen',
 'abgeändert',
 'umformulieren',
 'neu formulieren',
 'updaten',
 'Update',
 'Verbesserung',
 'ausgebessert',
 'aufdatieren',
 'Bearbeitung',
 'Neuformulierung',
 'richtig gestellt',
 'Berichtigung',
 'verbessern',
 'korrigiert',
 'Überarbeitung',
 'Neufassung',
 'überarbeitet',
 'upgedated',
 'beheben',
 'redigiert',
 'Umformulierung',
 'Aktualisierung',
 'berichtigt',
 'Korrektion',
 'neu formuliert',
 'Revision',
 'geändert',
 'verbessert',
 'überprüfen',
 'Fehlerverbesserung',
 'revidieren',
 'Nachbesserung',
 'überholt',
 'ausbessern',
 'Nachprüfung',
 'überarbeiten',
 'umformuliert',
 'Richtigstellung',
 'Korrektur',
 'bearbeiten',
 'Änderung',
 'redigieren',
 'Anpassung',
 'berichtigen']

In [41]:
# Okay, da waren 10 Schlüssel doppelt belegt. Daher die riesige Anzahl an Matches im ersten Durchgang. 

In [65]:
# 2 Ansätze: 

# Easy Approach: Das Matching ist simpel, jedes Dokument wird in Sätze zerlegt.

# Hard Approach: Das Matching wird komplex, man extrahiert keine Sätze sondern die Postion des 
# matches +- eine Zeichenanzahl. Text kann dabei in Form von Rohdaten bleiben. Vorteil: 
# Man übersieht nicht aus versehen Matches (z.B.) unter Grafiken (sind keine Sätze).

#=============================================================================================
# 1. EASY APPROACH
#=============================================================================================

# Pfadliste
pfade = Path.fetch("test_data")

for file in pfade:
    print(file)
    
    # Jahresabschluss einlesen
    parsedPDF = parser.from_file(file)

    # Name des Dokuments speichern
    name = os.path.basename(file)[0:-4] #letzter Block um ".pdf" abzuschneiden
    
    # Leere Keyword-Variable initiieren
    keyword_match = ''
    
    # Einzelne Seiten aufrufen
    pages_raw = parsedPDF_test['content'].split('\n\n\n\n')
    pages = [page for page in pages_raw if len(page)]
    
    # Seitenzähler innitiieren
    page_number = 0
    
    for page in pages:
        
        page_number += 1
        
        # Text_prep: Absätze (\n) und Tabs (\t) raus; Liste aus Sätzen erstellen // Normalerweise schriebt man noch alles klein, aber wegen KWL nicht möglich. 
        text = page.replace('\n', "").replace('\t', "").strip().lower()
        satz_liste = text.split('.')

        # Satzweise die gesamte KWL checken lassen. Wenn Match, dann Satz m it Nummer in Match_Liste ablegen
        satz_counter = 0
        for satz in satz_liste:
            satz_counter += 1
            for key in KWL:
                if re.search(key, satz):
                    with open("./arbeitsprobe.csv", "a", encoding="utf-8") as File:
                        File.write(f"{name},{page_number},{satz_counter},{key},{satz}\n")


C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Bayer AG2009_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\BMW AG2008_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Deutsche Post AG2010_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\E.ON AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Evonik Industries AG2011_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Fraport AG FFM Airport2012_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Henkel AG_Co.KGAA2013_SR.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\Linde AG_SR_2014.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\MAN SE_SR_2015.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\test_data\RWE AG_SR_2016.pdf
C:\Users\test\Documents\GitHub\hhu_whk_project_restatements\t

In [49]:

    
# Jahresabschluss einlesen
parsedPDF_test = parser.from_file('C:\\Users\\test\Documents\\GitHub\\hhu_whk_project_restatements\\test_data\\Bayer AG2009_SR.pdf')




Volkswagen AG_SR_2018


In [62]:
clean_pages = [page for page in pages if len(page)]
clean

['\n\nNachhaltigkeitsbericht 2009\n\n\nDie Publikation richtet sich an Geschäftspartner und \nMitarbeiter ebenso wie an Aktionäre, Nichtregie-\nrungsorganisationen, Lieferanten, Behörden und die \nbreite Öffentlichkeit. Die Anliegen unserer Stakehol-\nder bilden die Grundlage unserer Berichterstattung \nund unserer Nachhaltigkeitsstrategie, die wir 2009 \naktualisiert haben. \n\nMit dieser Strategie legen wir den Fokus unseres \nNachhaltigkeitsengagements auf die Gesundheits-\nversorgung, die Ernährungssicherung und den Klima- \nund Ressourcenschutz. In diesen Bereichen können \nwir aufgrund unserer Kompetenzen die effi zientesten \nund wirksamsten Lösungsbeiträge leisten. Gleich-\nzeitig besitzen sie eine hohe geschäftliche Relevanz \nfür unser Unternehmen. Detaillierte Informationen \nzur Bayer-Nachhaltigkeitsstrategie und ihren neuen \nSchwerpunkten  stellen wir im folgenden Bericht \nim Kapitel „Strategie & Management“ sowie in den \nSchwerpunktkapiteln vor. Aktuelle Informationen 

In [ ]:
#=============================================================================================
# 1. HRAD APPROACH
#=============================================================================================